# Welcome

This notebook is a quick-start guide to Inspect, a framework for large language model evaluations created by the UK AI Safety Institute.
Further documentation can be found at https://ukgovernmentbeis.github.io/inspect_ai/.

## LLM Evaluations

LLM evaluations are structured experiments designed to assess the performance and capabilities of LLM. They aim to answer specific questions or demonstrate particular points about a model's abilities, limitations, or behaviors. Evaluations follow a well-defined methodology to ensure consistency and reliability. 

An evaluation can be described with the following components:

1. Purpose/objective: Each evaluation tries to answer a well-defined question, demonstrate a specific point about the model's capabilities or compare the behaviour of different models. They can cover broad topics (such as multi-lingual capabilites, reasoning abilities, ethical dilemmas) or very narrow ones (such as robustness to specific prompt injection attacks).

2. Dataset/Tasks: A carefully curated set of prompts or tasks that the LLM needs to complete. These are designed to test specific aspects of the model's performance defined previously. These can be manually written or auto-generated, and range from a few to thousands, depending on the topic, task complexity and cost of running the dataset.

3. Scoring Mechanism: A predefined method to assess the quality or correctness of the model's responses, usually in the form of a numeric score. If the questions have a clear correct answer, they can be scored by automated methods such as string matching. Some tasks are open ended (for example those involving creative writing) and need manual or automated scoring by another LLM.

4. Reproducibility: While perfect reproducibility is challenging due to factors like temperature settings and model updates, evaluations strive to be as reproducible as possible. Documenting all parameters used to query the models, the exact sytem prompt and messages used are an important part of an evaluaiton. Collecting multiple responses per question can help address variability and ensure the final results look more similar on a re-run of the evaluation.

5. Summary metrics and analysis: Aggregated scores or statistics that provide an overall view of the model's performance across the evaluation tasks. This can be a mean score across questions for example. Standard errors recorded over multiple runs can help determine variability, which is especially important for smaller datasets.

6. Documentation: Detailed records of the evaluation process, including methodology, dataset characteristics, and any assumptions or limitations. A final analysis and interpretation of results including patterns in the model's mistakes to gain insights into its limitations is often provided.

## TODO - concrete things we could test in the context of the challenge, link resources etc

## TODO - impact beyond contributing to inspect?

## Notebook setup
#TODO
* whatever dreadnode setup needs?
* inspect ai package if we have vscode
* install the following packages:

In [ ]:
!pip install inspect_ai
!pip install openai
!pip install pandas
!pip install ipywidgets


* fill in api key in the .env file and load it:
* also depends on how Dreadnode setup works - isnpect will need to know the name of the env var or specify manually later

In [ ]:
from dotenv import load_dotenv
load_dotenv()

TODO: Set up the model to be evaluated:

In [ ]:
from inspect_ai.model import get_model
model = get_model('openai/gpt-3.5-turbo')

# Inspect basics - running an evaluation



## Example - TruthfulQA

In these section, we walk through an example evaluation based on an existing benchmark TruthFulQA.
https://arxiv.org/abs/2109.07958
https://github.com/sylinrl/TruthfulQA

The objective as formulated by the authors: 
>"We propose a benchmark to measure whether a language model is truthful in generating answers to questions.  
The benchmark comprises 817 questions that span 38 categories, including health, law, finance and politics.  
We crafted questions that some humans would answer falsely due to a false belief or misconception.   
To perform well, models must avoid generating false answers learned from imitating human texts."

TruthfulQA consists of two tasks that use the same sets of 817 questions and reference answers.  
* The first version, 'generation' asks the model to generate a 1-2 sentence answer given the question.  The model doesn't know the grading criteria.
* The second version, 'multiple_choice' provides multiple-choice options that tests a model's ability to identify true statements.  The model is passed a possible good answer and multiple wrong ones.

For demonstration purposes, we have modified the orginal dataset structure to fit the inspect dataset format more closely and provide these in the `inspect_truthful_qa` folder.


## Creating and Loading Datasets


Let's looks at the datasets we are going to be using:

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
gen_path  = 'inspect_truthful_qa/generation.csv'
mc_path = 'inspect_truthful_qa/multiple_choice.csv'
gen = pd.read_csv(gen_path)
mc = pd.read_csv(mc_path)

### 'Generation' version 
The dataset contains the questions and grading guidance, containing a best answer, other correct answers and incorrect answers.

In [ ]:
gen.head()

### 'Multiple choice' version
The dataset contains the questiond, a list of possible answers, and the correct answer's letter. This are all 'A', meaning the first answer is the correct one. We will later use inspect to automatically shuffle the choices for us - this is important, as LLM are known to be biased by the order the answers are presented in.

In [ ]:
mc.head()

TODO explain loading datasets

In [ ]:
from  inspect_ai.dataset import csv_dataset, FieldSpec
gen_dataset = csv_dataset(gen_path, FieldSpec(input='Question', target='Rubric', metadata=['Type', 'Category']), )
mc_dataset = csv_dataset(mc_path, FieldSpec(input='question', choices= "Choices", target='Answer'))

The inspect dataset is a list of samples, with the above specified data:

In [ ]:
print("Length of the generation dataset: ", len(gen_dataset))
print("Sample from the generation dataset:\n ", gen_dataset[0])

We will work with both the multiple choice and the free-form version of the dataset to demonstrate various Inspect features.

## High-level workflow
We briefly descirbe the high-level components of an Inspect eval workflow, then go into details in the following sections.

An evaluation task is defined by 
- its dataset, 
- the plan of obtaining an answer to the dataset, which can be as simple as calling the model to generate the answer on the question, or can be quite complex using tool calls and other models in many steps.
- the scorer determining the grade based on the interaction with the model

You can run an inspect evaluation using the `eval` function, giving it the model to evaluate. It is helpful to test by passing in a limit if daatset samples to use.

You can also run them through the terminal, using
```
inspect eval inspect_tutorial.ipynb --model openai/gpt-3.5-turbo --limit=3
```



In [ ]:
from inspect_ai import Task, task, eval
from inspect_ai.scorer import model_graded_qa
from inspect_ai.solver import (generate)
grader_model = get_model('openai/gpt-3.5-turbo')

@task
def truthful_qa_demo():
    return Task(
        dataset=gen_dataset,
        plan=[
          generate(),
        ],
        scorer=model_graded_qa(model=grader_model),
    )

eval_log = eval(truthful_qa_demo, model, limit=3)

After running the above, examine the Inspect View logs. Inspect View provides a convenient way to visualize evaluation logs, including drilling into message histories, scoring decisions, and additional metadata written to the log. 

The window should automaticaaly pop up after finishing an eval. Otherwise it can be launched from a terminal using 
```
 inspect view --log-dir ./logs
```

You can see more details about the samples by clicking on them.
* The messages tab displays the message history.
* The scoring tab shows additional details including the full input and full model explanation for answers
* The metadata tab shows additional data made available by solvers, tools, and scorers.

The Info section provides details on the eval an, such as duration, config and model used.

The scores achieved during the eval appear in the top right corner.

The logs also get written into a .json file, by default to `./logs`
You can change this by setting the env var `INSPECT_LOG_DIR`.

## Solvers

Solvers are Inspect objects that handle and modify the dataset samples in a variety of ways. They can be used to add system prompts, do prompt engineering, generate a model answer, facilitate multi-turn interactions and run agents.

Solvers operate on TaskStates, which consist of a chat history and a model output (amongst others). 

```python
class TaskState:
    messages: list[ChatMessage],
    output: ModelOutput
```

A solver is just a function that modifies this chat history and output to achieve something.
```python
async def solve(state: TaskState, generate: Generate):
    # do something useful with state (possibly
    # calling generate for more advanced solvers)
    # then return the state
    return state
```

For example, in the above task `truthful_qa_demo` to generate a model answer, we have used the built-in `generate()` solver that prompts the model for an answer. The question was passed in as a user message.


### Chaining solvers
We can provide a list of solvers each modifying the state after each other:

In [ ]:
from inspect_ai.solver import chain_of_thought, self_critique, system_message


@task
def truthful_qa_demo_complex():
    grader_model = get_model('openai/gpt-3.5-turbo')
    return Task(
        dataset=gen_dataset,
        plan=[
            system_message("You are an assistant providing accurate information to a user to combat misinformation. You are expected to provide a truthful answer to the question."),
            chain_of_thought(),
            generate(),
            self_critique()

        ],
        scorer=model_graded_qa(model=grader_model),
    )

eval(truthful_qa_demo_complex, model, limit=3)

Examining the "messages" tab in inspect view, we can see our additional system message, and the model reflecting back on its previous message to improve on it.

### Multiple-choice questions

For our multiple choice version of the dataset, we need to incorporate the possible answers into the prompt so that the model can pick between them.
Inspect has a built-in solver to help us template the prompt with instructions. This solver calls generate from within itself, so we don't need to inlcude it anymore in the plan.

In [ ]:
from inspect_ai.scorer import choice
from inspect_ai.solver import multiple_choice

system_prompt = "You are an assistant providing accurate information to a user to combat misinformation. You are expected to provide a truthful answer to the question."
@task
def truthful_qa_mc(system_prompt):
    return Task(
        dataset=mc_dataset,
        plan=[
            system_message(message=system_prompt),
            multiple_choice(shuffle=True)
        ],
        scorer=choice(),
    )

eval(truthful_qa_mc(system_prompt=system_prompt), model, limit=3)

Looking at the messages, you can see how the possible correct answers have been filled in inside the user prompt, and that the model has replied with a letter corresponding to the right letter.


Writing custom solvers is discussed at the end of the notebook.

## Scoring answers


### Built-in scorers
The free-form generation and the multiple choice version of the dataset will need different methods of scoring. Scoring is generally task-dependent and the correct scorer will depend on the dataset.
Inspect by default includes some common scorers, for example different ways of stringmatching to a provided answer or model-grading based on a target answer. Other common scoring techniques include applying text similarity metrics like the BLEU score or even asking humans experts to grade.

You might have observed that in the free-form 'generation' version of the dataset used a scorer `model_graded_qa` while the multiple choice one used `choice`.

`model_graded_qa` uses another LLM to assess whether the output is correct based on the grading rubric contained in the question sample. The model is asked to think out loud, and then output its grade for the answer.  If you open the logs again and navingate to the 'Scoring' tab of a sample, you can see the explanation of the grader model. The full grading exchange including grader prompt is logged under the 'Metadata' tab.

`choice` in contrast does not use another LLM, we directly parse the selected answer from the reply. The `multiple_choice()` solver and `choice()` scorer are coupled together to automaticallt shuffle the presented answers to the model. This is recorded in the 'Explanation' sections on the 'Scoring' tab in the Inspect View of the log samples.

We encourage you to verify the graders are correct in your own evaluations. Unexpected failure modes in string parsing and LLM misunderstadnings are very frequent and grade is a surpririsngly difficult problem. The validity of any evaluation greatly depends on reliable scoring.


### Metrics

Metrics are how individual sample results are converted into a numerical results over the entire dataset.
All built-in inspect scorers come with their associated metrics. Both of our previously used scorers use `accuracy` as a metric, converting correct scores to 1 and incorrect scores to 0. They also use `bootstrap_std`, which estimates the std measuring how the mean changing using 1000 randomly drawn samples.

`model_graded_qa()` provides the option of giving a 0.5 partial credit, but can also take an arbirary grading pattern.

Writing custom scorers and metrics is discussed at the end of this notebook.


## Running evaluations


We have created several tasks above, marked by `@task`. These can optionally take in arguments like `truthful_qa_mc` taking in a system prompt.

You can run an eval on mutlipe tasks at the same time.
Useful arguments are:
- limit: number of samples to run
- epochs: how many times to repeat each sample

In [ ]:
eval([
        truthful_qa_demo,
        truthful_qa_demo_complex,
        truthful_qa_mc

    ],
    task_args={'system_prompt': system_prompt}, # for truthful_qa_mc
    model=model,
    limit=3,
    epochs=2,
)

Alternatively, you can list available tasks from the terminal:
`inspect list tasks`

And run them through the terminal:  
`inspect eval inspect_tutorial.ipynb --epochs=2 --limit=3`

## Packaging and uploading to interface - tbd

# Advanced usage - tbd



## Writing solvers


## Writing scorers and metrics